INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "LAW-bert-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/dataset/LAW-bert-20210411-0832


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/law.tar.gz
!mkdir data_bin
!tar -xvf  'law.tar.gz' -C 'data_bin'

--2021-04-11 08:32:17--  https://github.com/hoangtrungchinh/clc_data/raw/master/law.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/law.tar.gz [following]
--2021-04-11 08:32:17--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/law.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 988814 (966K) [application/octet-stream]
Saving to: ‘law.tar.gz’

law.tar.gz          100%[===================>] 965,64K  5,47MB/s    in 0,2s    

2021-04-11 08:32:19 (5,47 MB/s) - ‘law.tar.gz’ saved [988814/988814]

_law.vi
_law.en


In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/_law.en']
['data_bin/_law.vi']
17967
17967


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

14373
14373
1797
1797
1797
1797
A person aged from fifteen years to below eighteen years may be an authorized representative, except where the law provides for that the civil transaction must be entered into and performed by a person who has reached eighteen years of age.

Người từ đủ mười lăm tuổi đến chưa đủ mười tám tuổi có thể là người đại diện theo uỷ quyền, trừ trường hợp pháp luật quy định giao dịch dân sự phải do người từ đủ mười tám tuổi trở lên xác lập, thực hiện.

The Law on Bankruptcy is applied to the bankruptcy process of every enterprise and cooperative based in the territory of Vietnam.

Luật Phá sản được áp dụng khi giải quyết phá sản đối với doanh nghiệp, hợp tác xã được thành lập trên lãnh thổ Cộng hòa xã hội chủ nghĩa Việt Nam.

A lawsuit is instituted over an administrative decision or act of a provincial-level People’s Committee or provincial-level People’s Committee chairperson which is related to many subjects;

Khiếu kiện quyết định hành chính, hành vi hành chí

In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 6672
drwxrwxr-x 3 chinh chinh    4096 Thg 4  11 08:32 .
drwxrwxr-x 5 chinh chinh    4096 Thg 4  11 08:32 ..
drwxrwxr-x 2 chinh chinh    4096 Thg 4  11 08:32 data_bin
-rw-rw-r-- 1 chinh chinh  257314 Thg 4  11 08:32 en_test
-rw-rw-r-- 1 chinh chinh 2096540 Thg 4  11 08:32 en_train
-rw-rw-r-- 1 chinh chinh  265528 Thg 4  11 08:32 en_valid
-rw-rw-r-- 1 chinh chinh  988814 Thg 4  11 08:32 law.tar.gz
-rw-rw-r-- 1 chinh chinh  314903 Thg 4  11 08:32 vi_test
-rw-rw-r-- 1 chinh chinh 2560025 Thg 4  11 08:32 vi_train
-rw-rw-r-- 1 chinh chinh  325494 Thg 4  11 08:32 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  176.0082974433899


# Calculate similarity

In [9]:
import faiss
vec_lst_srcfile_copy = vec_lst_srcfile.copy()
start = time.time()
index = faiss.IndexFlatIP(vec_lst_srcfile_copy.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile_copy)
index.add(vec_lst_srcfile_copy)
k = 6
distance, index = index.search(vec_lst_srcfile_copy, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  1.60097336769104
14373
14373
[[    0  4092  3389  4120  8037  3532]
 [    1  1610  9343 11153  6364  7317]
 [    2  4401  4307  1745  4738 11962]
 ...
 [14370  4992  6763  5200  6329 11081]
 [14371  5816 13903  8185 13816 10946]
 [14372  8951  1254  1865  1823  4448]]
[[1.         0.8861637  0.88371193 0.8821244  0.8461986  0.84168524]
 [1.0000002  0.79404676 0.70592886 0.6897241  0.6869191  0.68546885]
 [1.0000001  0.91459054 0.80838156 0.73446584 0.7079896  0.68380123]
 ...
 [1.0000002  0.8021705  0.77634656 0.76792455 0.7593357  0.7446659 ]
 [0.99999994 0.8738327  0.85086924 0.8413041  0.81209445 0.78889924]
 [1.0000002  0.90267485 0.7501807  0.7475535  0.72425145 0.70952046]]


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dt

count = 0
for i in range(len(lst_srcfile)):
    j = 1 #1 is index of biggest simmilarity
    score = distance[i][j]
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    sim_res = dot_product / (norm_a * norm_b)

    if score >=0.98 :
        count+=1
        print("==========================")
        for m in range(len(index[i])):
            print("--------", distance[i][m])
        print(i, index[i][j], distance[i][j], sim_res, 1-dt.cosine(a,b))
        print(lst_srcfile[i])
        print(lst_srcfile[index[i][j]])
        print("==========================")
print(count)

-------- 1.0000001
-------- 0.98919064
-------- 0.93834925
-------- 0.91291046
-------- 0.8922404
-------- 0.83196706
58 2207 0.98919064 0.9891905 0.9891904592514038
If this offence poses a threat to life, health or property of other people and is not promptly prevented, the offender shall be liable to a fine of from VND 10,000,000 to VND 50,000,000 or face a penalty of up to 02 years' community sentence or 03 - 24 months' imprisonment.

If this offence poses a threat to life, health or property of other people and is not promptly prevented, the offender shall be liable to a fine of from VND 10,000,000 to VND 100,000,000 or face a penalty of up to 02 years' community sentence or 03 - 24 months' imprisonment.

-------- 1.0000001
-------- 0.98282456
-------- 0.9554142
-------- 0.9537979
-------- 0.9142717
-------- 0.90782714
70 627 0.98282456 0.98282427 0.982824444770813
To perform other duties and exercise other powers in making accusations in accordance with the Criminal Procedure Code

-------- 0.9999997
-------- 0.99891746
-------- 0.6968181
-------- 0.6309375
-------- 0.62974924
-------- 0.6209015
5062 549 0.99891746 0.99891764 0.9989176392555237
Where the offeror dies or lacks of legal capacity or has limited cognition and behavior control after the offeree has replied accepting the offer, the offer to enter into a contract shall still be valid, unless the contents of contract is associated with the personal identity of the offeror.

Where the offeree dies or lacks of legal capacity or has limited cognition and behavior control after the offeree has replied accepting the offer, the offer to enter into a contract shall still be valid, unless the contents of contract is associated with the personal identity of the offeror.

-------- 1.0000001
-------- 0.98767686
-------- 0.88761437
-------- 0.8254239
-------- 0.79269636
-------- 0.75996107
5083 1436 0.98767686 0.9876766 0.987676739692688
Full names and addresses of residence places or work places of the judgments de

-------- 1.0000001
-------- 0.9921497
-------- 0.9082668
-------- 0.8948748
-------- 0.8639073
-------- 0.8468397
9916 10341 0.9921497 0.9921494 0.992149293422699
Name, head office address and enterprise identification number;

Name, enterprise identification number and head office address;

-------- 1.0
-------- 0.9998419
-------- 0.91780305
-------- 0.91320664
-------- 0.86260396
-------- 0.8538
9981 4139 0.9998419 0.99984187 0.9998418092727661
The offence results in property damage of from VND 200,000,000 to under VND 1,000,000,000;

The offence results in property damage of from VND 200,000,000 to under VND 1,000,000,000.

-------- 1.0000002
-------- 0.99946666
-------- 0.9072668
-------- 0.9016809
-------- 0.8992531
-------- 0.8978956
10198 1514 0.99946666 0.99946666 0.9994664788246155
The offence results in bodily harm to ≥ 03 people who suffer from a total physical disability of ≥ 61%;

The offence results in bodily harm to ≥ 03 people who suffer from a total physical disability

In [11]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        j=5
        score = distance[i][j]
        content = ""
        content_label_arr = []
        if score >= threshold and score <0.999:
            num_gth_thres += 1
            best_simi_index = index[i][j]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres*100/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 2655 / 14373 18.472135253600502 %
threshold = 0.85 EM sentences = 781 / 14373 5.43379948514576 %
threshold = 0.9 EM sentences = 190 / 14373 1.321923050163501 %
threshold = 0.95 EM sentences = 20 / 14373 0.13914979475405273 %
=== Ending, Total time (second):  0.9478471279144287
